In [1]:
# import necessary libraries
import pandas as pd
import sidetable
from Kleffy import kleffy_utility as ku

# set pandas dataframe global options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# initialise variables
data_dir = 'data'
doctors_fn = 'doctors.csv'
hospitals_fn = 'hospital_visits.csv'
patients_fn = 'patients.csv'

In [3]:
# read/load csv files
doctors = ku.read_file(filename=doctors_fn, path=data_dir)
hospital_visits = ku.read_file(filename=hospitals_fn, path=data_dir)
patients = ku.read_file(filename=patients_fn, path=data_dir)

In [4]:
# merge hospital visits dataframe and paients dataframe into one - intermediate step
hvp =  ku.merge_dataframe(df1=hospital_visits, df2=patients, left_on='patient_id', right_on='id')

In [5]:
# merge hvp dataframe and doctors dataframe into one - final step
sdf =  ku.merge_dataframe(df1=hvp, df2=doctors, left_on='doctor_id', right_on='id', 
                                right_suffix='doctor', 
                                drop_rcolumns=['created_at'])

In [6]:
# Feature engineering: create year, month, and day_name columns from created_at
single_df = ku.create_year_month_day_column(dataframe=sdf, date_column='created_at')

In [7]:
# ku.save_as_csv(hvp, 'hospital_visits_patients', 'Output')

In [8]:
# save final single dataframe
ku.save_as_csv(df=single_df, filename='single_records', path='Output')

single_records.csv saved successfully!


## Number of hospital visits per day

In [9]:
hvd = pd.DataFrame(single_df.groupby('created_at').size(), 
                    columns=['Number of visits per day'])
hvd.index.name = 'date of visit'

In [10]:
hvd.head(10)

,Number of visits per day
date of visit,
2021-05-01,13
2021-05-02,10
2021-05-03,28
2021-05-04,13
2021-05-05,31
2021-05-06,22
2021-05-07,25
2021-05-08,10
2021-05-09,8


## Number of patients attended to per doctor per month showing their names

In [16]:
# method 1
df1 = single_df.groupby(['doctor_id','month', 'name']).agg(dict(patient_id='count')).stb.subtotal().rename(columns={'patient_id':'patient count'})
df1['patient count'] = df1['patient count'].astype(int)
df1.head(20)

patient count
doctor_id                month name                                                       
3TB6T8CT/m/sBtYXy8XpTg== July  1vbeI+TzP2v2CHMaWSdVpSrbwzihSJv//7eqjZk2nOs=              1
                               5CyNu5QdzU2WJCA+hskS9XZQxHZOA+FhIEpjlT9+jtQ=              1
                               9TuSI17gA+HXhKeh4ibaJJwWY6CNXqgICDFsMHUunPk=              1
                               GKQINGs2vns7Ocrd9x/FAMAm6jdgzGUNgPFtdjrI8g4=              1
                               JBobAv4PTG4yh+FFtt4p20cVxsY3wBA+rmbT5zqD2QE=              1

## The ratio of female to male patient visits per month

In [179]:
gender_count=single_df.groupby(['month', 'sex'])['sex'].count()
gender_total=single_df.groupby(['month'])['sex'].count()
ratio=((gender_count/gender_total)*100).round(2)
ratio = pd.DataFrame(ratio).rename(columns={'sex':'ratio'})

In [180]:
ratio

ratio
month sex          
July  female  57.66
      male    42.34
June  female  58.75
      male    41.25
May   female  61.25
      male    38.75